# Creando wrappers de modelos para integrarlos con *Scikit-Learn*


En este notebook, se mostrará cómo crear un wrapper para un modelo de `Flux` en Julia y usarlo en un `Pipeline` junto con `GridSearchCV` para encontrar los mejores hiperparámetros. Este procedimiento es idéntico para integrar cualquier tipo de modelo. Simplemente es necesario tener en cuenta dos consideraciones. 

La primera es que el modelo debe de implementar el interface de *ScikitLearnBase*, o al menos como se verá más adelante aquellas funciones necesarias para poder ejecutarse.

La segunda de las consideraciones es que, dado que en Julia Sciki-Learn es un wrapper a su vez del paquete de Python, una vez que se llame a dicho paquete, no se puede volver a Julia y todos los modelos deberan de devolver los resultados a Julia antes de proseguir. Por poner un ejemplo claro, si intetásemos usar GridSearchCV con de Python, el modelo en FLux no podría integrarse ya que los resultados intermedios no son accesibles desde Python a la matriz de Julia. En cambio, si se usa como se verá más adelante el paquete GrisSearchCV implementado en Julia, si que se obtienen los resultados de los paquetes Python y por tanto funcionará correctamente.


## 1. Instalación de Paquetes

El primero de los puntos es asegurarnos de que la librería `Flux`, que es una biblioteca de aprendizaje automático para Julia, está instalada. 

``` julia
using Pkg
Pkg.add(["Flux", "ScikitLearn", "ScikitLearnBase", "Statistics"])
```

In [2]:
import ScikitLearnBase: BaseClassifier, fit!, predict, score, @declare_hyperparameters, is_classifier # Se explica más adelante
using Flux
using Flux.Losses
using Statistics

## 2. Creación de una estructura contenedora

Para la creación del *wrapper* lo primero en Python sería crear una clase que corrobore un cierto *interface*, es decir un conjunto de firmas de métodos que nos permitan hacer llamadas de manera uniforme y reconocible para la librería ScikitLearn.

En Julia a diferencia de Python u otros lenguajes Orientados a Objetos, no existen las clases como tal. Pero no es un problema ya que se puede simular con una esructura mutable. Estas estructuras permiten cambiar los valores y almacenar llamadas a diferentes métodos, de manera similar a una clase, además  servira de tipo para la sobrecarga de los métodos.

El primer paso es la importación del interface, en este caso, `ScikitLearnBase` que  aportará todas las firmas de métodos que se van a necesitar. En este caso véase un ejemplo con las redes de classificación.

In [3]:
mutable struct ClassANN <: BaseClassifier
    # Hiperparámetros del modelo (no aprendidos de los datos)
    topology::AbstractVector{Int}
    transferFunctions::AbstractVector{Function}
    maxEpochs::Int
    minLoss::Real
    learningRate::Real

    # Parámetros aprendidos (modelo de Flux y optimizador)
    model::Chain
    opt::ADAM

    # Constructor que acepta los hiperparámetros como argumentos con nombre
    ClassANN(; topology=[1], transferFunctions=fill(σ, 1), maxEpochs=1000, minLoss=0.0, learningRate=0.01) =
        new(topology, transferFunctions, maxEpochs, minLoss, learningRate, Chain(), ADAM(learningRate))
end


En dicho código se observa una primera parte de parámetros que serán los necesarios para el constructor, y que será responsabilidad del usuario definir. A continuación se encuentran parámetros utilitarios que serán extraidos de los datos pero que no se expondrán al exterior y finalmente la definición del constructor de esta "pseudo-clase". Esta última cuenta con los valores por defecto del conjunto.

Para finalizar la definición se hace necesario definir dos cosas adiciones, la primera un método que declara el tipo de problema que es capaz de resolver, en el caso del ejemplo de clasificación. Para ello es necesaria la siguiente llamada

In [4]:
# Indicar que ClassANN es un clasificador
is_classifier(::ClassANN) = true

is_classifier (generic function with 3 methods)

El segundo de los elementos necesarios es declarar aquellos parámetros que es necesario aportar por parte del usuario para construir una instancia de este tipo

In [5]:
@declare_hyperparameters(ClassANN, [:topology, :transferFunctions, :maxEpochs, :minLoss, :learningRate])

## 3. Fuciones auxiliares
A continuación se definirían todas las funciones auxiliares necesarias para la ejecución del modelo, esto se puede hacer dentro del las propias llamadas a `fit!`, `predict` y `score`. Otra opción es utilizar las ya implementadas en ocasiones anteriores como pueden ser las implementadas en la asignaturad e Fundamentos de Aprendizaje Automático para ello se puede emplear la siguiente llamada

``` julia
    include("MipaqueteFunciones.jl")

```
Sin embargo por una cuestión de autocontención a continuación se pone como ejemplo un código de las funciones necesarias para este ejemplo.

In [7]:
#Funcion para realizar la codificacion, recibe el vector de caracteristicas (uno por patron), y las clases
function oneHotEncoding(feature::AbstractArray{<:Any,1}, classes::AbstractArray{<:Any,1})
    # Primero se comprueba que todos los elementos del vector esten en el vector de clases (linea adaptada del final de la practica 4)
    @assert(all([in(value, classes) for value in feature]));
    numClasses = length(classes);
    @assert(numClasses>1)
    if (numClasses==2)
        # Si solo hay dos clases, se devuelve una matriz con una columna
        oneHot = reshape(feature.==classes[1], :, 1);
    else
        # Si hay mas de dos clases se devuelve una matriz con una columna por clase
        # Cualquiera de estos dos tipos (Array{Bool,2} o BitArray{2}) vale perfectamente
        # oneHot = Array{Bool,2}(undef, length(targets), numClasses);
        #oneHot =  BitArray{2}(undef, length(feature), numClasses);
        oneHot = classes'.== features
    end;
    return oneHot;
end;

# Esta funcion es similar a la anterior, pero si no es especifican las clases, se toman de la propia variable
oneHotEncoding(feature::AbstractArray{<:Any,1}) = oneHotEncoding(feature, unique(feature));

# Sobrecargamos la funcion oneHotEncoding por si acaso pasan un vector de valores booleanos
#  En este caso, el propio vector ya está codificado, simplemente lo convertimos a una matriz columna
oneHotEncoding(feature::AbstractArray{Bool,1}) = reshape(feature, :, 1);

In [8]:
#Función para la construcción de la Red Neuronal Artificial
function buildClassANN(numInputs::Int, topology::AbstractVector{Int}, numOutputs::Int;
            transferFunctions::AbstractVector{Function}=fill(σ, length(topology)))
    layers = []
    numInputsLayer = numInputs

    for (i, numNeurons) in enumerate(topology)
        push!(layers, Dense(numInputsLayer, numNeurons, transferFunctions[i]))
        numInputsLayer = numNeurons
    end

    if numOutputs == 1
        push!(layers, Dense(numInputsLayer, 1, σ))
    else
        push!(layers, Dense(numInputsLayer, numOutputs, identity))
        push!(layers, softmax)
    end

    return Chain(layers...)
end

buildClassANN (generic function with 1 method)

También nos podría hacer falta la función `trainANNClass`pero en este caso, por razones meramente formativas, se hará una implementación sencilla en la propia función para ejemplificarlo

## 4. Implementar las funciones de *Scikit-Learn*

Como ya se indicó, es necesario implemtar los métodos requeridos por ScikitLearnBase, en el caso de la clasificació, `fit!`, `predict`, y `score`.

En primer lugar, la función `fit!` para realizar el ajuste con el uso de Flux para el entrenamiento del modelo, en caso de tenerlo separado en una función se podría emplear.

In [ ]:
# Implementar el ajuste (fit!) del modelo
function fit!(model::ClassANN, X, y)
    numInputs = size(X, 2)
    numOutputs = length(unique(y))

    # Construir el modelo usando Flux
    model.model = buildClassANN(numInputs, model.topology, numOutputs, transferFunctions=model.transferFunctions)
    model.opt = ADAM(model.learningRate)

    # Convertir las etiquetas a formato one-hot si es clasificación multiclase
    if numOutputs > 1
        y = oneHotEncoding(y, unique(y))
    end

    # Definir la función de pérdida
    loss(x, y) = Flux.crossentropy(model.model(x), y)

    # Entrenar el modelo
    # Alternativamente
    #(model, results) = trainANNClass(model.topology, (X', y')], model.maxEpoachs, model.minLoss, model.learningRate)
    for epoch in 1:model.maxEpochs
        Flux.train!(loss, Flux.params(model.model), [(X', y')], model.opt)
        current_loss = loss(X', y')
        println("Epoch: $epoch, Loss: $current_loss")
        if current_loss <= model.minLoss
            break
        end
    end
    return model
end

Se empleará tambié la función de `predict`

In [ ]:
# Implementar la predicción (predict) del modelo
function predict(model::ClassANN, X)
    if size(model.model(X'), 1) > 1
        return Flux.onecold(model.model(X'), 1:size(model.model(X'), 1))
    else
        return round.(model.model(X'))
    end
end

Y la función de `score`porque vamos a usar este *wrapper* dentro de un `GridSearchCV`

In [ ]:
# Función adicional para calcular el puntaje (score) del modelo
function score(model::ClassANN, X, y)
    predictions = predict(model, X)
    return mean(predictions .== y)
end

### 4.1 Transformadores y regresión

En caso de emplearse un wrapper de otro método como puede ser un PCA o alguna técnica de aprendizaje no supervisado. Sería necesrio implementar las funciones `fit!`, `transform` y `fit_transform`.
POr su parte en el caso de implementar un modelo de regresión, sería preciso implementar `fit!`, `predict` `score`y en la mayoría de los casos `predict_proba`. Consulte el API en [Scikit Learn Model API](https://scikitlearnjl.readthedocs.io/en/latest/api/)

# 5. La interacción con las librería

El código anterior puede salvarse en un fichero y cargarlo con una un include o mediante un módulo que se emplee a posteriori, en dicho caso acuerdese de hacer el export de las funciones necesarias del módulo.

En todo caso en el código siguiente también puede ver como llamar esas funciones y como se integran dentro de un `Pipeline` se puede cambiar para buscar la mejor combinación

In [ ]:
# Fichero Test.jl

using ScikitLearn
using ScikitLearn.Pipelines: Pipeline, named_steps
using ScikitLearn.GridSearch: GridSearchCV         #IMPORTANTE usar la implementación en Julia
#@sk_import model_selection: GridSearchCV          # Esta es la implementación en Python y dará error al no encontrar el Wrapper

@sk_import decomposition: PCA
@sk_import datasets: load_iris

# Cargar los datos
iris = load_iris()
X = iris["data"]
y = iris["target"]

# Definir la búsqueda en cuadrícula (hiperparámetros a probar)
param_grid = Dict(
    "ann__maxEpochs" => [500, 1000], 
    "ann__learningRate" => [0.01, 0.1]
)

#Crear una instancia de ClassANN
# Definir los parámetros
topology = [3, 4]
functions = [σ, σ]
maxEpochs = 1000
minLoss = 0.0
learningRate = 0.01

# Crear una instancia de ClassANN
ann = ClassANN(topology=topology, transferFunctions=functions, maxEpochs=maxEpochs, minLoss=minLoss, learningRate=learningRate)

estimators = [("pca",PCA()),("ann",ann)]
pipe = Pipeline(estimators)

# Configurar el GridSearchCV
grid_search = GridSearchCV(pipe, param_grid)

# Ajustar el modelo usando GridSearchCV
fit!(grid_search, X, y)

# Obtener el mejor modelo y sus hiperparámetros
println("Mejor modelo: ", grid_search.best_estimator_)
println("Mejores hiperparámetros: ", grid_search.best_params_)